<a href="https://colab.research.google.com/github/DikRoman/DikRoman/blob/main/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


### Загрузка датасета

При помощи pandas загрузим необходимые данные в память и посмотрим на данные

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Мы имеем: 891 пассажира

Каждый объект представлен 12 признаками: ID пассажира, Имя, пол, номер билета, возраст и т.д.
Survived - это целевое значение, означает выжил пассажир или не выжил.
- количество детей пасажиров Parch

Таким образом, нам нужно решить задачу бинарной классификации: понять, выжил пассажир или не выжил. Научиться предсказывать, вышел но или нет.

## 1.1. Предобработка данных и признаков

### 1.1.1. Понять, что лишнее

Необходимо удалить ненужные признаки, которые в нашем понимании не могут влиять на то, выжил человек или нет, которые будут только мешать нашей обучающейся модели. ID, номер билета и имя вряд ли могут повлиять на выживаемость.

In [5]:
data = data.drop(['PassengerId','Ticket', 'Name'], axis=1)  #axis = 1 - для обозначения того, что нужно удалять колонки

Для начала отделим колонку целевой переменной от датасета и разделим датасет на тренировочный и тестовый. На тренировочном датасете будем обучать KNN, а на тренировочном - тестировать.

Запишем матрицу признаков в целевой столбец в разные перменные дата и Y. 

In [6]:
y = data['Survived']

In [7]:
data = data.drop(['Survived'], axis=1)

Взглянем еще раз на типы данных получившегося датасета

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Cabin     204 non-null    object 
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 55.8+ KB


Видим, что в трех колонках у нас данные указанны в виде строк (типа object). Модель умеет работать только с числовыми данными.

Также проверим датасет на пропуски.

1. Займемся заполнение пропусков

In [9]:
data.columns[data.isna().any()].tolist()

['Age', 'Cabin', 'Embarked']

1.1. Age. Возраст человека может иметь возраст от 1 допустим, до 120. В данном случае нам важно чтобы возраст не выбивался из общей статистики.

In [10]:
data['Age'].mean()

29.69911764705882

In [11]:
data['Age'].median()

28.0

Заполним пропуски значением медианы

In [12]:
data['Age'] = data['Age'].fillna(28)

### 1.2. Embarked

Колонка Embarked имеет категориальный признак, поэтому для нее как раз имеет смысл заполнить пропуски наиболее встречающимся значением.

In [13]:
data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [14]:
data['Embarked'] = data['Embarked'].fillna('S')

Взглянем на колонку Cabin.

In [15]:
data['Cabin'].isna().sum()

687

Всего 687 пропусков в сравнении со всем датасетом  -- это довольно много. заполнение проупсков будет очень неточным. С учетом того, что признак возможно не очень полезен, удалим колонку.

In [16]:
data = data.drop('Cabin', axis=1)

In [17]:
assert not data.isnull().values.any() #выдает ошибку если в данных остались пропуски

Перевод категориальных признаков в числовые

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


После обработки датасета у нас осталось всего 2 категориальных признака: Пол и Embarked.

In [19]:
data["Sex"] = data["Sex"].astype('category') #меняем тип столбца на категориальный

In [20]:
data["Sex"] = data["Sex"].cat.codes #кодируем

Готово, мы преобразовали типы пола в категорию 1 и 0.

Embarked

In [21]:
data["Embarked"].describe()

count     891
unique      3
top         S
freq      646
Name: Embarked, dtype: object

Видим что в колонке Embarked всего 3 уникальных значения.
Решение: сделаем три столбца, каждый с названием -  значением колонки. Каждый столбец будет содержать значение 0 и 1. Для тех, если есть совпадение с названием - то 1. Если нет - то 0.


In [22]:
data = pd.get_dummies(data, columns = ["Embarked"]) #произведем бинаризацию функцией get_dummies

In [23]:
data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,22.0,1,0,7.2500,0,0,1
1,1,0,38.0,1,0,71.2833,1,0,0
2,3,0,26.0,0,0,7.9250,0,0,1
3,1,0,35.0,1,0,53.1000,0,0,1
4,3,1,35.0,0,0,8.0500,0,0,1


Предобработка завершена.

Осталось разбить данные на тренировочный и валидационный датасет и можно обучать модель.

In [24]:
''' Функция разбивает тестовую выборку по умному - т.е. чтобы данные на тренировочной и тестовой выборке примерно совпадали'''

from sklearn.model_selection import train_test_split

Тест сайз - 30% всех данных оставляем для валидации. Остальные 70% оставляем для обучения.

In [25]:
train_data, val_data, train_y, val_y = train_test_split(data, y, test_size = 0.3)

2. Обучение модели

In [26]:
from sklearn.neighbors import KNeighborsClassifier

'''модуль где лежат различные метрики'''

from sklearn.metrics import accuracy_score #функция вводящая метрику точности

In [27]:
knn = KNeighborsClassifier(n_neighbors = 3)  
#Все гиперпараметры вводятся в ммоент обяъвления модели knn. колво соседец в данном случае 3

In [28]:
#передаем данные модели для обучения
 
knn.fit(train_data, train_y)

KNeighborsClassifier(n_neighbors=3)

In [29]:
'''Попросим обученную модель выдать нам ответы на валидационные данные с помощью метода predict'''
predicted = knn.predict(val_data)
predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0])

Мы видим вектор ответов, один ответ на каждый объект из val_data.
Теперь выведем правильные ответы val_y на экран.

In [30]:
np.array(val_y)

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 0])

Визуально сравнивать ответы не эффективно, поэтому передадим ответы метрике score.

In [31]:
accuracy_score(predicted, val_y)

0.6828358208955224

Подбор гиперпараметра k

Теперь подберем параметр k - количества соседей алгоритма, чтобы алгоритм лучше обучился. и выдавал больше правильных ответов.
Мы переберем параметр k от 1 до 20 включительно, обучим для каждого значения к классификатор KNN и посчитаем метрику score на валидационной выборке.

In [32]:
val_scores = []
for k in range(1, 21):
    k = KNeighborsClassifier(n_neighbors=k) 
    knn.fit(train_data, train_y)
    predicted = knn.predict(val_data)
    acc_score = accuracy_score(predicted, val_y)
    
    val_scores.append(acc_score)

3. Получение ответов для тестового датасета.

In [35]:
test_data = pd.read_csv('test.csv')
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


Предобрабатываем тестовый датасет также, как тренировочный.

In [36]:
#удаляем колонки
test_data = test_data.drop(['PassengerId','Ticket', 'Name'], axis=1)

#заполняем пропуски
test_data['Age'] = test_data['Age'].fillna(28)
test_data['Embarked'] = test_data['Embarked'].fillna('S')
test_data = test_data.drop('Cabin', axis=1)

#переводим категории в числа и производим бинаризацию
test_data["Sex"] = test_data["Sex"].astype('category') #меняем тип столбца на категориальный
test_data["Sex"] = test_data["Sex"].cat.codes #кодируем
test_data = pd.get_dummies(test_data, columns = ["Embarked"]) #произведем бинаризацию функцией get_dummies

test_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,34.5,0,0,7.8292,0,1,0
1,3,0,47.0,1,0,7.0000,0,0,1
2,2,1,62.0,0,0,9.6875,0,1,0
3,3,1,27.0,0,0,8.6625,0,0,1
4,3,0,22.0,1,1,12.2875,0,0,1


Посмтрим, нет ли в тестовых данных пропусков, которых не былов  тренировочных.

In [37]:
test_data.isna().any()

Pclass        False
Sex           False
Age           False
SibSp         False
Parch         False
Fare           True
Embarked_C    False
Embarked_Q    False
Embarked_S    False
dtype: bool

In [38]:
#заполним пропуски медианным значением тренировочного датасета

test_data['Fare'] = test_data['Fare'].fillna(train_data['Fare'].median())

Обучим нашу модель KNN на всей тренировочной выборке с оптимальным параметром k (полученным при подборе параметров выше)

In [39]:
knn = KNeighborsClassifier(n_neighbors = 17)  
knn.fit(data, y)

KNeighborsClassifier(n_neighbors=17)

In [40]:
test_predicted = knn.predict(test_data)

In [41]:
test_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

Сохраним ответы на тестовый датасет в нужном виде для отправки на соревнование

In [42]:
test_predicted = pd.DataFrame({"Survived": test_predicted})
test_predicted["PassengerId"] = list(range(892, 892+len(test_data)))


test_predicted.to_csv("test_predicted.csv", index= False)

test_predicted

,Survived,PassengerId
0,0,892
1,0,893
2,0,894
3,0,895
4,0,896
...,...,...
413,0,1305
414,1,1306
415,0,1307
416,0,1308
